In [1]:
# General
# %pip install numpy
# %pip install scipy
# %pip install matplotlib
# %pip install soundfile
import numpy as np

# Signal Processing
from scipy import signal
from scipy.fftpack import fft, ifft

from IPython.display import Audio 
import soundfile as sf

import matplotlib.pyplot as plt

In [2]:
# implement 2nd order peaking filter to remove 573Hz tone here
x, Fs = sf.read("bad_audio.wav")
def second_order_peak(audio_arr, cutoff, bandwidth, sr, G):
    return_audio = np.zeros(len(audio_arr))
    wC = 2 * cutoff/sr
    wB = 2 * bandwidth/sr   
    v0 = 10 ** (G / 20)
    h0 = v0 - 1
    c = None
    if G >= 0:
        c = (np.tan(np.pi * wB/2)-1) / (np.tan(np.pi * wB/2)+1)
    else:
        c = (np.tan(np.pi * wB/2)-v0) / (np.tan(np.pi * wB/2)+v0)
    d = -np.cos(np.pi * wC)
    xH = [0,0]
    for i in range(len(audio_arr)):
        xH_new = audio_arr[i] - d*(1-c) * xH[0] + c*xH[1]
        ap_y = -c * xH_new + d*(1-c) * xH[0] + xH[1]
        xH = [xH_new, xH[0]]
        return_audio[i] = 0.5 * h0 * (audio_arr[i] - ap_y) + audio_arr[i]
    return return_audio

hp_audio = second_order_peak(x, 570, 1, Fs, -80)
# hp_audio *= 1000.0
Audio(hp_audio, rate = Fs)


In [3]:
# implement 2nd order high shelf filter to remove >14kHz noise here
def high_shelf(audio_arr, cutoff, sr, G):
    return_audio = np.zeros(len(audio_arr))
    wC = 2 * cutoff/sr
    v0 = 10 ** (G / 20)
    K = np.tan(np.pi * cutoff/sr)
    b0 = (v0 * (1 + np.sqrt(2) * K + K ** 2)) / (1 + np.sqrt(2) * K + v0 * K ** 2)
    b1 = 2 * (v0 * (K ** 2 - 1)) / (1 + np.sqrt(2 * v0) * K + v0 * K ** 2)
    b2 = (v0 * (1 - np.sqrt(2) * K + K ** 2)) / (1 + np.sqrt(2 * v0) * K + v0 * K ** 2)
    a1 = (2 * (v0 * K ** 2 - 1)) / (1 + np.sqrt(2 * v0) * K + v0 * K ** 2)
    a2 = (1 - np.sqrt(2 * v0) * K + v0 * K ** 2) / (1 + np.sqrt(2 * v0) * K + v0 * K ** 2)

    for x in range(len(audio_arr)):
        return_audio[x] = b0 * audio_arr[x] + b1 * audio_arr[x-1] + b2 * audio_arr[x-2] - a1 * return_audio[x-1] - a2 * return_audio[x-2]

    return return_audio

hs_audio = high_shelf(hp_audio, 13500, Fs, -65)

# Final Audio
Audio(hs_audio, rate = Fs)

1.
Original:
y[n] = x[n] + y[n-1]
Z transform:
y[z] = x[z] + (z^-1) * y[z]
y[z] - y[z] * (z^-1) = x[z]
Factor out y[z]:
y[z] * (1 - z^-1) = x[z]

Transfer function in terms of y(z)/x(z):

y(z) / x(z) = 1 / 1 - z^-1

or 
y(z) / x(z) = z / z - 1

2.
Poles: 
This transfer function has a pole at z = 1, on the unit circle, making it unstable. A pole on the unit circle indicates the output is potentially unbounded given certain inputs. This would not be a desirable filter as it would be too unstable, either feedbacking to generate a hum and likely failing to filter frequencies as desired.

3. Please see attached "filter diagram" jpeg. 




